In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
from sklearn.decomposition import IncrementalPCA, PCA
import numpy as np

In [ ]:
train_df = pd.read_csv('../input/train.tsv', sep='\t')

In [ ]:
train_df = train_df.iloc[:10000, :]

In [ ]:
test_df = pd.read_csv('../input/test.tsv', sep='\t')

In [ ]:
X = train_df[['item_condition_id','brand_name', 'shipping']]
y = train_df['price']

In [ ]:
X_t = test_df[['item_condition_id','brand_name', 'shipping']]

In [ ]:
X['brand_name'] = X['brand_name'].fillna('unknown')

In [ ]:
X_t['brand_name'] = X_t['brand_name'].fillna('unknown')

In [ ]:
brand_dummies = pd.get_dummies(X['brand_name'])

In [ ]:
brand_dummies_t = pd.get_dummies(X_t['brand_name'])

In [ ]:
brand_dummies['item_condition_id'] = X['item_condition_id']
brand_dummies['shipping'] = X['shipping']

In [ ]:
brand_dummies_t['item_condition_id'] = X_t['item_condition_id']
brand_dummies_t['shipping'] = X_t['shipping']

In [ ]:
pca = PCA(n_components=100, svd_solver='randomized')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(brand_dummies, y)

In [ ]:
brand_dummies_comps_train = pca.fit_transform(X_train)

In [ ]:
brand_dummies_comps_test = pca.transform(X_test)

In [ ]:
X_train_comps = brand_dummies_comps_train[:,:20]
X_test_comps = brand_dummies_comps_test[:,:20]

In [ ]:
regr = RandomForestRegressor(n_estimators=300)

In [ ]:
regr.fit(X_train_comps, y_train)

In [ ]:
regr.score(X_train_comps, y_train)

In [ ]:
regr.score(X_test_comps, y_test)

In [ ]:
r2_score(y_true=y_test, y_pred=regr.predict(X_test_comps))

In [ ]:
brand_dummies_t_choose_columns = brand_dummies_t[[col for col in X_train.columns if col in brand_dummies_t.columns]]

In [ ]:
brand_dummies_t_choose_columns.shape

In [ ]:
cols_to_add = [col for col in X_train.columns if col not in brand_dummies_t_choose_columns.columns]

In [ ]:
for col in cols_to_add:
    brand_dummies_t_choose_columns[col] = 0

In [ ]:
comps_to_predict = pca.transform(brand_dummies_t_choose_columns)

In [ ]:
y_pred_result = regr.predict(comps_to_predict[:, :20])

In [ ]:
result_df = pd.DataFrame({'price':y_pred_result})

In [ ]:
result_df = result_df.reset_index()

In [ ]:
result_df.rename(columns={'index': 'test_id'})

In [ ]:
result_df.to_csv('submission.csv')